In [3]:
from selenium import webdriver 
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()   
url = "https://docs.nvidia.com/cuda/"
driver.get(url)

In [3]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Function to scrape a webpage and its sub-links recursively
def scrape_page(url, depth, max_depth, visited_urls, data):
    if depth > max_depth or url in visited_urls:
        return

    visited_urls.add(url)

    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract content (modify as per your requirement)
        content = soup.get_text()
        data.append((url, content))

        # Extract links
        links = soup.find_all('a', href=True)
        for link in links:
            full_link = urljoin(url, link['href'])
            if 'https://docs.nvidia.com/cuda/' in full_link:
                scrape_page(full_link, depth + 1, max_depth, visited_urls, data)
    
    except Exception as e:
        print(f"Error scraping {url}: {e}")

# Main function to initiate scraping
def main():
    start_url = "https://docs.nvidia.com/cuda/"
    max_depth = 2  # Depth of sub-links to scrape
    visited_urls = set()
    scraped_data = []

    scrape_page(start_url, 0, max_depth, visited_urls, scraped_data)

    # Print some sample scraped data
    for url, content in scraped_data[:5]:  # Displaying first 5 entries for illustration
        print(f"URL: {url}")
        print(f"Content:\n{content[:200]}...\n")

if __name__ == "__main__":
    main()


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [8]:
#pip install selenium webdriver-manager
#pip install selenium webdriver-manager requests beautifulsoup4 sentence-transformers milvus transformers flask
#python your_script.py


SyntaxError: invalid syntax (687263809.py, line 4)

In [ ]:
# Assuming scraped_data is populated from previous step
# Simplified chunking (each paragraph as a chunk)
def chunk_data(data):
    chunks = []
    for url, content in data:
        paragraphs = content.split('\n\n')  # Split by paragraphs
        chunks.extend([(url, para.strip()) for para in paragraphs if para.strip()])
    return chunks

# Chunk the scraped data
chunked_data = chunk_data(scraped_data)

# Displaying first few chunks
for i, (url, chunk) in enumerate(chunked_data[:5]):
    print(f"Chunk {i + 1}: URL: {url}\n{chunk[:200]}...\n")


In [ ]:
from sentence_transformers import SentenceTransformer
from milvus import Milvus, DataType

# Initialize Milvus client
client = Milvus(uri='tcp://127.0.0.1:19530')

# Create a collection
collection_name = 'cuda_docs'
collection_param = {
    "fields": [
        {"name": "embeddings", "type": DataType.FLOAT_VECTOR, "params": {"dim": 768}},
        {"name": "metadata", "type": DataType.VARCHAR, "params": {"max_length": 1024}}
    ],
    "segment_row_limit": 4096,
    "auto_id": True
}
client.create_collection(collection_name, collection_param)

# Load pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to encode chunks into embeddings and insert into Milvus
def encode_and_insert_chunks(chunks):
    for url, chunk in chunks:
        embeddings = model.encode([chunk], show_progress_bar=False)[0]
        embeddings = embeddings.tolist()  # Convert numpy array to list

        # Insert into Milvus
        client.insert(collection_name, [{"embeddings": embeddings, "metadata": {"url": url, "chunk": chunk}}])

# Encode and insert chunks into Milvus
encode_and_insert_chunks(chunked_data)

# Flush changes to disk
client.flush([collection_name])


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/query', methods=['POST'])
def query_endpoint():
    data = request.json
    query = data.get('query', '')
    retrieved_docs = retrieve(query)
    answer = answer_question(query, retrieved_docs)
    return jsonify(answer)

if __name__ == '__main__':
    app.run(debug=True)
